In [3]:
import numpy as np
import pandas as pd
import math
import sys
import tqdm

In [15]:
gru4rec_15 = pd.read_csv('/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general/output/1_EASE_top15.csv')
ease_15 = pd.read_csv('/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general/output/GRU4Rec_top15.csv')

In [16]:
file_path = ['/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general/output/1_EASE_top15.csv',
             '/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general/output/GRU4Rec_top15.csv']

In [63]:

def ensemble_double_ratio(args):
    
# ratio1 - Top1-k : Topk-15
    print("Write ratio for Top1-10 and Top11-15. Sum of ratios must be 1.\nex) 0.75 0.25")
    inputs = input().strip()
    top_weight_list = [float(item) for item in inputs.split() if inputs]

    # exception handling
    if len(top_weight_list) != 2:
        ValueError("Wrong ratios. Try again.\n")
    if math.fabs(sum(top_weight_list) - 1.0) > sys.float_info.epsilon:
        ValueError("Sum of ratios is not 1. Try again.\n")



    # ratio2 - 모델별 weight 
    print("\nWrite ratio for input csv files. Sum of ratios must be 1.\nex) 0.5 0.5")
    inputs = input().strip()
    model_weight_list = [float(item) for item in inputs.split() if inputs]

    # exception handling
    if len(model_weight_list) < 2:
        ValueError("Wrong ratios. Try again.\n")
    if math.fabs(sum(model_weight_list) - 1.0) > sys.float_info.epsilon:
        ValueError("Sum of ratios is not 1. Try again.\n")



    # summary inputs
    print("\n----------input information----------")
    print(f"Ratio of Top1-k & Topk-15:\n{top_weight_list[0]} : {top_weight_list[1]}")
    print("Ratio of input csv files:")
    for idx in range(len(file_path)):
        print(f"[{idx}] : {file_path[idx]} - {model_weight_list[idx]}")
    print("-------------------------------------\n")


    top1015 = pd.read_csv(file_path[0]).groupby("user").apply(lambda x: x.iloc[:10,1]).reset_index().drop('level_1', axis=1)
    top1015.head(10)

    print(top_weight_list,model_weight_list# top 1,2
        )

    file_list = []
    for path in file_path:
        csv = pd.read_csv(path)
        file_list.append(csv)

    user_list = file_list[0]["user"].unique()

    file_path



    # voting
    '''
    ex)
        model A : model B = 0.6 : 0.4
        top1-10 : top11-15 = 0.75 : 0.25

    1. model A & Top1-10 = 0.6 x 0.75 ~= 0.45
    2. model A & Top10-15 = 0.6 x 0.25 ~= 0.15
    3. model B & Top1-10 = 0.4 x 0.75 = 0.3
    4. model B & Top10-15 = 0.4 x 0.25 = 0.1
    '''
    #user_list = file_list[0]["user"].unique()
    file_list = []
    for m_id, path in enumerate(file_path):

        top0110 = pd.read_csv(path).groupby("user").apply(lambda x: x.iloc[:10,1]).reset_index().drop('level_1', axis=1)
        top1115 = pd.read_csv(path).groupby("user").apply(lambda x: x.iloc[10:15,1]).reset_index().drop('level_1', axis=1)

        top0110['weight'] = model_weight_list[m_id] * top_weight_list[0]
        top1115['weight'] = model_weight_list[m_id] * top_weight_list[1]
        file_list.append(top0110)
        file_list.append(top1115)

    #get user information


    #print(len(file_list))

    result = pd.concat(file_list).reset_index(drop=True)
    #result[result['user']==11]


    g = result.groupby(['user','item']).agg({'weight': 'sum'})
    g['weight'] = - g['weight']
    g = g.sort_values(['user','weight'])
    g['weight'] = - g['weight']
    result = g.groupby('user').head(10).reset_index()

    return result[['user','item']]



In [50]:
file_list

[          user   item  weight
 0           11   4370   0.375
 1           11   4886   0.375
 2           11  40815   0.375
 3           11     47   0.375
 4           11  32587   0.375
 ...        ...    ...     ...
 313595  138493   5349   0.375
 313596  138493   8970   0.375
 313597  138493  32587   0.375
 313598  138493   4022   0.375
 313599  138493  53125   0.375
 
 [313600 rows x 3 columns],
           user   item  weight
 0           11   2987   0.125
 1           11   2174   0.125
 2           11   2329   0.125
 3           11  37386   0.125
 4           11   3996   0.125
 ...        ...    ...     ...
 156795  138493   1022   0.125
 156796  138493  48394   0.125
 156797  138493   3000   0.125
 156798  138493    293   0.125
 156799  138493   2324   0.125
 
 [156800 rows x 3 columns],
           user   item  weight
 0           11   7438   0.375
 1           11  30707   0.375
 2           11  63082   0.375
 3           11  63876   0.375
 4           11   8961   0.375
 ...      

In [43]:
csv = file_list[0]
idx = 0 
add = 0
csv["item"][:]

0          4370
1          4886
2         40815
3            47
4         32587
          ...  
470395     1022
470396    48394
470397     3000
470398      293
470399     2324
Name: item, Length: 470400, dtype: int64

In [40]:
# top_weight_list,model_weight_list
movie_list = []
idx = 0
for user in user_list:
    tmp_dict = dict()
    for i, path in enumerate(file_list):
        
        for add in range(10):
            if csv["item"][idx + add] not in tmp_dict:
                tmp_dict[csv["item"][idx + add]] = top_weight_list[0] * model_weight_list[i]
            else:
                tmp_dict[csv["item"][idx + add]] += top_weight_list[0] * model_weight_list[i]
        for add in range(10, 15):
            if csv["item"][idx + add] not in tmp_dict:
                tmp_dict[csv["item"][idx + add]] = top_weight_list[1] * model_weight_list[i]
            else:
                tmp_dict[csv["item"][idx + add]] += top_weight_list[1] * model_weight_list[i]
    sorted_dict = sorted(tmp_dict.items(), key=lambda item: item[1], reverse=True)
    for i in range(10):
        movie_list.append(sorted_dict[i][0])
    idx += 15

In [41]:
movie_list

[7438,
 30707,
 63082,
 63876,
 8961,
 6870,
 48780,
 41997,
 8873,
 56367,
 1029,
 2273,
 4014,
 3256,
 914,
 3039,
 2539,
 3082,
 1287,
 2724,
 65130,
 64622,
 60950,
 64957,
 63082,
 61323,
 63876,
 62293,
 26237,
 64575,
 3052,
 1380,
 4022,
 353,
 2617,
 4034,
 4995,
 1288,
 6874,
 1641,
 81229,
 55995,
 53993,
 56152,
 6294,
 79293,
 60126,
 71057,
 37830,
 7161,
 7293,
 4246,
 4308,
 48385,
 5218,
 45720,
 5299,
 7451,
 27706,
 8529,
 597,
 56367,
 4896,
 5989,
 46578,
 4995,
 1097,
 8360,
 4022,
 500,
 3751,
 1641,
 1732,
 1380,
 1653,
 3147,
 4027,
 2640,
 2324,
 2700,
 1694,
 1950,
 3271,
 2966,
 1132,
 4881,
 4326,
 3730,
 3260,
 3551,
 53125,
 2628,
 6539,
 480,
 1721,
 4308,
 4306,
 30793,
 8360,
 3793,
 34523,
 33880,
 36537,
 45728,
 2596,
 4873,
 30810,
 5577,
 4979,
 27904,
 3556,
 7371,
 7323,
 5791,
 56367,
 2291,
 6711,
 8784,
 44694,
 60950,
 2321,
 1035,
 3176,
 2657,
 2081,
 5060,
 3052,
 1954,
 1028,
 1148,
 4993,
 1196,
 1198,
 1210,
 33794,
 4226,
 2571,
 5855

In [ ]:
gru4rec_15[gru4rec_15['user']==11]

### 기타


In [2]:
ease = pd.read_csv('/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general/output/1_EASE_epoch10_@.csv')
sas = pd.read_csv('/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general/output/SASRec_@.csv')

In [5]:
gru4rec = pd.read_csv('/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general/output/GRU4Rec_$.csv')

In [7]:
gru4rec.shape

(313600, 2)

In [19]:
ngcf = pd.read_csv('/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general/output/NGCF.csv')


In [3]:
gru4rec = pd.read_csv('/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general/output/GRU4Rec.csv')


FileNotFoundError: [Errno 2] No such file or directory: '/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general/output/GRU4Rec.csv'

In [23]:
gru4rec.shape

(311197, 2)

In [21]:
ngcf.shape

(313600, 2)

In [28]:
sas.shape

(311471, 2)

In [4]:
gru4rec.groupby(['user']).count()

NameError: name 'gru4rec' is not defined

In [7]:
sas_user_count=sas.groupby(['user']).count().describe()
sas_user_count

,item
count,31359.000000
mean,9.932428
std,0.508484
min,1.000000
25%,10.000000
50%,10.000000
75%,10.000000
max,10.000000


In [4]:
sas[sas['user']==11]


,user,item
0,11,48780
1,11,8961
2,11,44665
3,11,7438
4,11,4886
5,11,32587
6,11,3996
7,11,293


In [14]:
ease['prior'] = [i for i in range(8)]*31360
ease.head(10)

,user,item,prior
0,11,4886,0
1,11,4370,1
2,11,8961,2
3,11,40815,3
4,11,2329,4
5,11,32587,5
6,11,37386,6
7,11,47,7
10,14,1198,0
11,14,2011,1


In [4]:
ease8 = ease.groupby('user').head(8)
ease8.head(10)

,user,item
0,11,4886
1,11,4370
2,11,8961
3,11,40815
4,11,2329
5,11,32587
6,11,37386
7,11,47
10,14,1198
11,14,2011


In [5]:
bert['prior'] = [i for i in range(8,18)]*31360

In [6]:
result = pd.concat([ease8, bert])
result

,user,item,prior
0,11,4886,NaN
1,11,4370,NaN
2,11,8961,NaN
3,11,40815,NaN
4,11,2329,NaN
...,...,...,...
313595,138493,593,13.0
313596,138493,4848,14.0
313597,138493,4022,15.0
313598,138493,53125,16.0


In [7]:
result = result.drop_duplicates(['user','item'],keep='first')
result

,user,item,prior
0,11,4886,NaN
1,11,4370,NaN
2,11,8961,NaN
3,11,40815,NaN
4,11,2329,NaN
...,...,...,...
313589,138492,1210,17.0
313592,138493,293,10.0
313596,138493,4848,14.0
313597,138493,4022,15.0


In [8]:
result = result.sort_values(['user','prior'])

In [9]:
result = result.groupby('user').head(10)
result

,user,item,prior
4,11,1704,12.0
6,11,3996,14.0
8,11,1127,16.0
0,11,4886,NaN
1,11,4370,NaN
...,...,...,...
313591,138493,2628,NaN
313592,138493,593,NaN
313593,138493,110,NaN
313594,138493,1270,NaN


In [ ]:
result[['user','item']].to_csv('./output/ratio_ensemble.csv',index=False)

In [3]:
import sys
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
import argparse
import os
import glob
import pdb

In [31]:


file_list=['/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general/output/1_EASE_epoch10_@.csv',
           '/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general/output/2_EASE_newversion.csv']




In [27]:
ratios = input(f"(모델 수 : {len(file_list)})띄어쓰기로 구분하여 ratio 입력하기")
ratios = ratios.split(" ")
ratios = list(map(int,ratios))


In [28]:
ratios

[3, 8]

In [30]:
pwd

'/opt/ml/git/04.for_ensemble/level2_movierecommendation-recsys-02/Recbole/general'

In [55]:
ratios = input(f"(모델 수 : {len(file_list)})띄어쓰기로 구분하여 ratio 입력하기")
ratios = ratios.split(" ")
ratios = list(map(int,ratios))
if len(ratios) != len(file_list):
    raise ValueError("모델 수와 가중치값의 수가 맞지 않음") 
df_list = [pd.read_csv(f) for f in file_list]

for df,ratio in zip(df_list,ratios):
    df["weight"] = ratio
result = pd.concat(df_list).reset_index(drop=True)
g = result.groupby(['user','item']).agg({'weight': 'sum'})
g['weight'] = - g['weight']
g = g.sort_values(['user','weight'])
g['weight'] = - g['weight']
result = g.groupby('user').head(10)
result = result.reset_index()

,user,item,weight
0,11,47,11
1,11,3996,11
2,11,4370,11
3,11,4886,11
4,11,8961,11
...,...,...,...
313595,138493,5349,11
313596,138493,293,8
313597,138493,4022,8
313598,138493,4848,8


In [58]:
result = result.reset_index()


In [59]:
result[['user','item']]

,user,item
0,11,47
1,11,3996
2,11,4370
3,11,4886
4,11,8961
...,...,...
313595,138493,5349
313596,138493,293
313597,138493,4022
313598,138493,4848


In [15]:

ratios = input(f"(모델 수 : {len(file_list)})띄어쓰기로 구분하여 ratio 입력하기")
ratios = ratios.split(" ")
ratios = list(map(int,ratios))
if len(ratios) != len(file_list):
    raise ValueError("모델 수와 가중치값의 수가 맞지 않음") 
df_list = [pd.read_csv(f) for f in file_list]

for df,ratio in zip(df_list,ratios):
    df["weight"] = ratio
result = pd.concat(df_list).reset_index(drop=True)
g = result.groupby('user').agg({'weight': 'sum', 'item': lambda x: x[1]})

g['weight'] = - g['weight']
g = g.sort_values(['user','weight'])
g['weight'] = - g['weight']
result = g.groupby('user').head(10)
result = result.sort_values(by='user')
result

KeyError: 1